### 1. 서울시 경찰서

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#데이터 불러오기
import pandas as pd
police = pd.read_csv('/content/drive/MyDrive/서울시_경찰서 현황.csv')
print(police.head())

      시도경찰청     위치     경찰서명칭            경찰서주소
0  서울특별시경찰청  서울특별시   서울중부경찰서        중구 수표로 27
1  서울특별시경찰청  서울특별시   서울종로경찰서       종로구 율곡로 46
2  서울특별시경찰청  서울특별시  서울남대문경찰서      중구 한강대로 410
3  서울특별시경찰청  서울특별시  서울서대문경찰서     서대문구 통일로 113
4  서울특별시경찰청  서울특별시   서울혜화경찰서  종로구 창경궁로 112-16


In [5]:
police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도경찰청   31 non-null     object
 1   위치      31 non-null     object
 2   경찰서명칭   31 non-null     object
 3   경찰서주소   31 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [6]:
# 카카오API를 사용하여 주소->좌표 변환
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  headers = {"Authorization": "KakaoAK 162c7473d8d38b7454be72d06227c98b"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

#테스트
crd = get_location("서울시 중구 을지로 234")
print(crd)

{'lat': '37.5664734170529', 'lng': '127.003943654119'}


In [7]:
#[도로명 주소->좌표 변환 함수]

import pandas as pd
import requests
import json

# 카카오 API를 사용하여 주소 -> 좌표 변환 함수
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK 162c7473d8d38b7454be72d06227c98b"}  # REST API 키 입력
    response = requests.get(url, headers=headers)
    api_json = json.loads(response.text)

    # 검색 결과가 없을 경우 처리
    if not api_json['documents']:
        return None  # 결과 없음 처리

    address_info = api_json['documents'][0]['address']
    # 위도와 경도를 딕셔너리 형태로 반환
    return {"lat": address_info['y'], "lng": address_info['x']}

In [8]:
#[도로명 주소->좌표로 변환 후 실행결과 저장]

# 위도를 구해서 '위도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['경찰서주소'])  # A 열의 주소를 입력
    if coords:
        return f"{coords['lat']}"  # 위도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police['위도'] = police.apply(add_coordinates, axis=1)  # 좌표를 구해 '위도' 열에 저장

# 경도를 구해서 '경도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['경찰서주소'])  # A 열의 주소를 입력
    if coords:
        return f"{coords['lng']}"  # 경도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police['경도'] = police.apply(add_coordinates, axis=1)  # 좌표를 구해 '경도' 열에 저장

# 결과 출력
print(police)

       시도경찰청     위치     경찰서명칭               경찰서주소                위도  \
0   서울특별시경찰청  서울특별시   서울중부경찰서           중구 수표로 27  37.5636917755092   
1   서울특별시경찰청  서울특별시   서울종로경찰서          종로구 율곡로 46  37.5758763395858   
2   서울특별시경찰청  서울특별시  서울남대문경찰서         중구 한강대로 410  37.5548113505249   
3   서울특별시경찰청  서울특별시  서울서대문경찰서        서대문구 통일로 113  37.5650853221302   
4   서울특별시경찰청  서울특별시   서울혜화경찰서     종로구 창경궁로 112-16  37.5717676404547   
5   서울특별시경찰청  서울특별시   서울용산경찰서       용산구 원효로89길 24  37.5415376293887   
6   서울특별시경찰청  서울특별시   서울성북경찰서         성북구 보문로 170  37.5897433363624   
7   서울특별시경찰청  서울특별시  서울동대문경찰서    동대문구 약령시로 21길 29  37.5851576234405   
8   서울특별시경찰청  서울특별시   서울마포경찰서        마포구 마포대로 183  37.5508289853671   
9   서울특별시경찰청  서울특별시  서울영등포경찰서       영등포구 국회대로 608  37.5259893677858   
10  서울특별시경찰청  서울특별시   서울성동경찰서        성동구 왕십리광장로 9  37.5620026859035   
11  서울특별시경찰청  서울특별시   서울동작경찰서        동작구 노량진로 148  37.5127958577043   
12  서울특별시경찰청  서울특별시   서울광진경찰서         광진구 자양로 167  37.5429114164928   
13  서울

In [9]:
#[좌표 이용->동 변환 함수]

def get_dong(lat, lng):
    try:
        url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lng}&y={lat}"
        headers = {"Authorization": "KakaoAK 162c7473d8d38b7454be72d06227c98b"}  # API 키 주의
        response = requests.get(url, headers=headers)

        if response.status_code == 200:  # 요청 성공
            result = response.json()
            for document in result['documents']:
                if document['region_type'] == 'B':  # region_type이 'B'인 경우
                    return document['region_3depth_name']
            return None  # 'B' 타입이 없는 경우
        else:
            return None  # 요청 실패 시 None 반환
    except Exception as e:
        print(f"Error: {e}")
        return None

In [10]:
# 각 행에 대해 '동' 정보를 추가
police['동'] = police.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)

#결과 출력
print(police)

       시도경찰청     위치     경찰서명칭               경찰서주소                위도  \
0   서울특별시경찰청  서울특별시   서울중부경찰서           중구 수표로 27  37.5636917755092   
1   서울특별시경찰청  서울특별시   서울종로경찰서          종로구 율곡로 46  37.5758763395858   
2   서울특별시경찰청  서울특별시  서울남대문경찰서         중구 한강대로 410  37.5548113505249   
3   서울특별시경찰청  서울특별시  서울서대문경찰서        서대문구 통일로 113  37.5650853221302   
4   서울특별시경찰청  서울특별시   서울혜화경찰서     종로구 창경궁로 112-16  37.5717676404547   
5   서울특별시경찰청  서울특별시   서울용산경찰서       용산구 원효로89길 24  37.5415376293887   
6   서울특별시경찰청  서울특별시   서울성북경찰서         성북구 보문로 170  37.5897433363624   
7   서울특별시경찰청  서울특별시  서울동대문경찰서    동대문구 약령시로 21길 29  37.5851576234405   
8   서울특별시경찰청  서울특별시   서울마포경찰서        마포구 마포대로 183  37.5508289853671   
9   서울특별시경찰청  서울특별시  서울영등포경찰서       영등포구 국회대로 608  37.5259893677858   
10  서울특별시경찰청  서울특별시   서울성동경찰서        성동구 왕십리광장로 9  37.5620026859035   
11  서울특별시경찰청  서울특별시   서울동작경찰서        동작구 노량진로 148  37.5127958577043   
12  서울특별시경찰청  서울특별시   서울광진경찰서         광진구 자양로 167  37.5429114164928   
13  서울

In [13]:
police.to_csv('서울시 경찰서 법정동.csv', index=False, encoding='utf-8-sig')

### 2. 서울시 지구대, 파출소

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#데이터 불러오기
import pandas as pd
police1 = pd.read_csv('/content/drive/MyDrive/서울시_지구대 파출소 현황.csv')
print(police1.head())

   연번  시도청   경찰서 관서명   구분          전화번호                          도로명 주소
0   1  서울청  서울중부  을지  지구대  02-2279-1908               서울특별시 중구  을지로 234
1   2  서울청  서울중부  광희  지구대  02-2233-1444             서울특별시 중구  퇴계로 375-1
2   3  서울청  서울중부  약수  지구대  02-2234-8112      서울특별시 중구  동호로 5길 15, 약수지구대
3   4  서울청  서울중부  신당  파출소  02-2252-0435  서울특별시 중구  다산로 248 (신당동, 신당파출소)
4   5  서울청  서울중부  장충  파출소  02-2274-9003               서울특별시 중구  동호로 261


In [16]:
police1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      243 non-null    int64 
 1   시도청     243 non-null    object
 2   경찰서     243 non-null    object
 3   관서명     243 non-null    object
 4   구분      243 non-null    object
 5   전화번호    243 non-null    object
 6   도로명 주소  243 non-null    object
dtypes: int64(1), object(6)
memory usage: 13.4+ KB


In [17]:
#[도로명 주소->좌표로 변환 후 실행결과 저장]

# 위도를 구해서 '위도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['도로명 주소'])  # 열의 주소를 입력
    if coords:
        return f"{coords['lat']}"  # 위도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police1['위도'] = police1.apply(add_coordinates, axis=1)  # '도로명 주소' 열 데이터를 기준으로 좌표를 구해 '위도' 열에 저장

# 경도를 구해서 '경도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['도로명 주소'])  # A 열의 주소를 입력
    if coords:
        return f"{coords['lng']}"  # 경도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police1['경도'] = police1.apply(add_coordinates, axis=1)  # '도로명 주소' 열 데이터를 기준으로 좌표를 구해 '경도' 열에 저장

# 결과 출력
print(police1)

      연번  시도청   경찰서 관서명   구분          전화번호                          도로명 주소  \
0      1  서울청  서울중부  을지  지구대  02-2279-1908               서울특별시 중구  을지로 234   
1      2  서울청  서울중부  광희  지구대  02-2233-1444             서울특별시 중구  퇴계로 375-1   
2      3  서울청  서울중부  약수  지구대  02-2234-8112      서울특별시 중구  동호로 5길 15, 약수지구대   
3      4  서울청  서울중부  신당  파출소  02-2252-0435  서울특별시 중구  다산로 248 (신당동, 신당파출소)   
4      5  서울청  서울중부  장충  파출소  02-2274-9003               서울특별시 중구  동호로 261   
..   ...  ...   ...  ..  ...           ...                             ...   
238  239  서울청  서울수서  대치  지구대   02-538-1312              서울특별시 강남구  삼성로 203   
239  240  서울청  서울수서  수서  파출소   02-445-0112            서울특별시 강남구  밤고개로1길 48   
240  241  서울청  서울수서  대왕  파출소  02-3411-0323              서울특별시 강남구  헌릉로 623   
241  242  서울청  서울수서  일원  지구대  02-3411-0112       서울특별시 강남구  일원로3길 35 일원파출소   
242  243  서울청  서울수서  개포  지구대   02-571-0112        서울특별시 강남구  개포로 261(1268)   

                   위도                경도  
0    37.5664734170529

In [18]:
#잘못된 주소값이 있는지 확인

def filter_rows_by_value(dataframe, column_name, value):
    """
    특정 열에서 주어진 값과 일치하는 행을 반환하는 함수.

    Args:
        dataframe (pd.DataFrame): 데이터프레임
        column_name (str): 열 이름
        value: 찾을 값 (A)

    Returns:
        pd.DataFrame: 조건에 맞는 행만 포함하는 데이터프레임
    """
    filtered_rows = dataframe[dataframe[column_name] == value]
    return filtered_rows

# '위도' 열에서 값이 'A'인 행만 필터링
rows_with_None = filter_rows_by_value(police1, '위도', 'None')
print(rows_with_None)

Empty DataFrame
Columns: [연번, 시도청, 경찰서, 관서명, 구분, 전화번호, 도로명 주소, 위도, 경도]
Index: []


In [19]:
# 각 행에 대해 '동' 정보를 추가
police1['동'] = police1.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)

#결과 출력
print(police1)

      연번  시도청   경찰서 관서명   구분          전화번호                          도로명 주소  \
0      1  서울청  서울중부  을지  지구대  02-2279-1908               서울특별시 중구  을지로 234   
1      2  서울청  서울중부  광희  지구대  02-2233-1444             서울특별시 중구  퇴계로 375-1   
2      3  서울청  서울중부  약수  지구대  02-2234-8112      서울특별시 중구  동호로 5길 15, 약수지구대   
3      4  서울청  서울중부  신당  파출소  02-2252-0435  서울특별시 중구  다산로 248 (신당동, 신당파출소)   
4      5  서울청  서울중부  장충  파출소  02-2274-9003               서울특별시 중구  동호로 261   
..   ...  ...   ...  ..  ...           ...                             ...   
238  239  서울청  서울수서  대치  지구대   02-538-1312              서울특별시 강남구  삼성로 203   
239  240  서울청  서울수서  수서  파출소   02-445-0112            서울특별시 강남구  밤고개로1길 48   
240  241  서울청  서울수서  대왕  파출소  02-3411-0323              서울특별시 강남구  헌릉로 623   
241  242  서울청  서울수서  일원  지구대  02-3411-0112       서울특별시 강남구  일원로3길 35 일원파출소   
242  243  서울청  서울수서  개포  지구대   02-571-0112        서울특별시 강남구  개포로 261(1268)   

                   위도                경도      동  
0    37.566473

In [25]:
police1.to_csv('서울시 파출소 지구대 법정동.csv', index=False, encoding='utf-8-sig')

### 3. 서울시 치안센터

In [22]:
#데이터 불러오기
import pandas as pd
police2 = pd.read_csv('/content/drive/MyDrive/서울시_치안센터 현황.csv')
print(police2.head())

   연번  시도청    경찰서     치안센터명                   주소
0   1  서울청   서울중부    충4치안센터       서울 중구 충무로2길 39
1   2  서울청   서울종로    구기치안센터        서울 종로구 비봉길 13
2   3  서울청  서울서대문   북아현치안센터  서울 서대문구 북아현로 59-1번지
3   4  서울청   서울혜화  종로3가치안센터       서울 종로구 돈화문로 28
4   5  서울청   서울용산    청파치안센터     서울 용산구 청파로 274-2


In [23]:
police2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      55 non-null     int64 
 1   시도청     55 non-null     object
 2   경찰서     55 non-null     object
 3   치안센터명   55 non-null     object
 4   주소      55 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.3+ KB


In [26]:
#[도로명 주소->좌표로 변환 후 실행결과 저장]

# 위도를 구해서 '위도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['주소'])  # A 열의 주소를 입력
    if coords:
        return f"{coords['lat']}"  # 위도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police2['위도'] = police2.apply(add_coordinates, axis=1)  # 좌표를 구해 '위도' 열에 저장

# 경도를 구해서 '경도' 열에 추가
def add_coordinates(row):
    coords = get_location(row['주소'])  # A 열의 주소를 입력
    if coords:
        return f"{coords['lng']}"  # 경도를 문자열로 저장
    else:
        return "No results"  # 결과가 없을 경우

police2['경도'] = police2.apply(add_coordinates, axis=1)  # 좌표를 구해 '경도' 열에 저장

# 결과 출력
print(police2)

    연번  시도청    경찰서     치안센터명                   주소                위도  \
0    1  서울청   서울중부    충4치안센터       서울 중구 충무로2길 39  37.5623991766563   
1    2  서울청   서울종로    구기치안센터        서울 종로구 비봉길 13  37.6095716272147   
2    3  서울청  서울서대문   북아현치안센터  서울 서대문구 북아현로 59-1번지  37.5626238022321   
3    4  서울청   서울혜화  종로3가치안센터       서울 종로구 돈화문로 28  37.5712443657225   
4    5  서울청   서울용산    청파치안센터     서울 용산구 청파로 274-2  37.5440912782433   
5    6  서울청   서울용산    용산치안센터        서울 용산구 신흥로 90  37.5461126719259   
6    7  서울청   서울용산   한남2치안센터      서울 용산구 이태원로 281   37.539770595227   
7    8  서울청   서울용산    이촌치안센터     서울 용산구 이촌로71길 24  37.5213567033653   
8    9  서울청   서울성북    삼선치안센터     서울 성북구 보문로29길 92  37.5862080046689   
9   10  서울청   서울성북   길음1치안센터   서울 성북구 정릉로 31길 30   37.6038806342175   
10  11  서울청   서울성북   정릉1치안센터      서울 성북구 동소문로 258  37.6038870030508   
11  12  서울청  서울동대문  답십리1치안센터       서울 동대문 고미술로 93  37.5646639481943   
12  13  서울청  서울동대문   장안3치안센터    서울 동대문구 장한로18길 78  37.5678516893414   
13  14

In [27]:
# 각 행에 대해 '동' 정보를 추가
police2['동'] = police2.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)

#결과 출력
print(police2)

    연번  시도청    경찰서     치안센터명                   주소                위도  \
0    1  서울청   서울중부    충4치안센터       서울 중구 충무로2길 39  37.5623991766563   
1    2  서울청   서울종로    구기치안센터        서울 종로구 비봉길 13  37.6095716272147   
2    3  서울청  서울서대문   북아현치안센터  서울 서대문구 북아현로 59-1번지  37.5626238022321   
3    4  서울청   서울혜화  종로3가치안센터       서울 종로구 돈화문로 28  37.5712443657225   
4    5  서울청   서울용산    청파치안센터     서울 용산구 청파로 274-2  37.5440912782433   
5    6  서울청   서울용산    용산치안센터        서울 용산구 신흥로 90  37.5461126719259   
6    7  서울청   서울용산   한남2치안센터      서울 용산구 이태원로 281   37.539770595227   
7    8  서울청   서울용산    이촌치안센터     서울 용산구 이촌로71길 24  37.5213567033653   
8    9  서울청   서울성북    삼선치안센터     서울 성북구 보문로29길 92  37.5862080046689   
9   10  서울청   서울성북   길음1치안센터   서울 성북구 정릉로 31길 30   37.6038806342175   
10  11  서울청   서울성북   정릉1치안센터      서울 성북구 동소문로 258  37.6038870030508   
11  12  서울청  서울동대문  답십리1치안센터       서울 동대문 고미술로 93  37.5646639481943   
12  13  서울청  서울동대문   장안3치안센터    서울 동대문구 장한로18길 78  37.5678516893414   
13  14

In [28]:
police2.to_csv('서울시 치안센터 법정동.csv', index=False, encoding='utf-8-sig')